In [ ]:
# adjust dyad position +/- 500
import General
from pathlib import Path

General.adjust_dyad_positions(
    dyad_file=Path('/media/cam/Data9/CortezAnalysis/Cam_calls/nucleosome_stuff/dyads.bed')
)

In [ ]:
# getfasta ran on the dyads to get the context
import BedtoolsCommands
from pathlib import Path
terminal_out, file_location = BedtoolsCommands.bedtools_getfasta(
    dyad_file=Path('/media/cam/Data9/CortezAnalysis/Cam_calls/nucleosome_stuff/dyads_files/dyads_plus-minus_1000.bed'),
    fasta_file=Path('/media/cam/Data9/CortezAnalysis/pipeline_files/hg19.fa')
)
print(terminal_out)

In [ ]:
import General
from pathlib import Path

General.filter_lines_with_n(
    dyad_fasta = Path('/media/cam/Data9/CortezAnalysis/Cam_calls/nucleosome_stuff/dyads_files/dyads_plus-minus_1000_hg19_fasta.fa'),
    dyad_bed=Path('/media/cam/Data9/CortezAnalysis/Cam_calls/nucleosome_stuff/dyads_files/dyads_plus-minus_1000.bed')
)


In [1]:
import General
from pathlib import Path

General.filter_acceptable_chromosomes(
    Path('/media/cam/Data9/CortezAnalysis/Cam_calls/8-oxo-G_Mapping_Data/split-reads/joined_bed/SRR_67-68_sorted_sorted.bed'), 'human'
    )

In [ ]:
# use linux to sort a file based on the first 3 and sixth column
import General
from pathlib import Path
# General.check_and_sort(Path('/media/cam/Data9/CortezAnalysis/Cam_calls/8-oxo-G_Mapping_Data/split-reads/joined_bed/SRR_67-68_sorted.bed'))
General.check_and_sort(Path('/media/cam/Data9/CortezAnalysis/Cam_calls/nucleosome_stuff/dyads_files/dyads_plus-minus_1000_filtered.bed'))

In [ ]:
# counts all the different positions in the dyad file
import multiprocessing as mp
import Statistics
from pathlib import Path

if __name__ == '__main__':
    mp.freeze_support()
    fasta_counter = Statistics.DyadFastaCounter(
        Path('/media/cam/Data9/CortezAnalysis/Cam_calls/nucleosome_stuff/dyads_files/dyads_plus-minus_1000_hg19_fasta_filtered.fa')
        )

In [ ]:
# counts all the different positions in the dyad file
import multiprocessing as mp
import Statistics
from pathlib import Path

if __name__ == '__main__':
    mp.freeze_support()
    fasta_counter = Statistics.MutationIntersector(
        mutation_file = Path('/media/cam/Data9/CortezAnalysis/Cam_calls/8-oxo-G_Mapping_Data/split-reads/joined_bed/SRR_67-68_sorted_sorted.bed'),
        dyad_file = Path('/media/cam/Data9/CortezAnalysis/Cam_calls/nucleosome_stuff/dyads_files/dyads_plus-minus_1000_filtered_sorted.bed')
    )

In [ ]:
import Statistics
from pathlib import Path
import pandas as pd
df = Statistics.df_division_and_standardization(
    mutation_counts=Path('/media/cam/Data9/CortezAnalysis/Cam_calls/8-oxo-G_Mapping_Data/split-reads/joined_bed/SRR_67-68_sorted_sorted_filtered_dyads_plus-minus_1000_filtered_sorted_intersected_mutations_counts.txt'),
    dyad_counts=Path('/media/cam/Data9/CortezAnalysis/Cam_calls/nucleosome_stuff/dyads_files/dyads_plus-minus_1000_hg19_fasta_filtered_counts.txt')
)
df.to_csv('test.txt', sep='\t', float_format='%.{}f'.format(4))